# DAN - Reuters

In [1]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
import keras
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Variables

In [2]:
num_classes = 8
SEED = 333
max_words = 10000

## Data loading

In [3]:
TRAIN_PATH = "../../data/data_reuters/r8-train-all-terms.txt"
TEST_PATH = "../../data/data_reuters/r8-test-all-terms.txt"


train = pd.read_csv(TRAIN_PATH, sep="\t", header=None)
test = pd.read_csv(TEST_PATH, sep="\t", header=None)

x_train_txt = train[1].values
y_train_txt = train[0].values
x_test_txt = test[1].values
y_test_txt = test[0].values

print(len(x_train_txt), 'train sequences')
print(len(x_test_txt), 'test sequences')

5485 train sequences
2189 test sequences


## Prep labels (y_test, y_train)

In [4]:
labels = sorted(set(test[0]))

y_test = [labels.index(l) for l in y_test_txt]
y_train = [labels.index(l) for l in y_train_txt]

In [5]:
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train = keras.utils.to_categorical(y_train, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (5485, 8)
y_test shape: (2189, 8)


### Split data

In [6]:
x_train80_txt, x_val_txt, y_train80, y_val = train_test_split(x_train_txt, y_train, test_size=0.2, random_state=SEED)

In [7]:
print('y_train80 shape:', y_train80.shape)
print('y_val shape:', y_val.shape)

y_train80 shape: (4388, 8)
y_val shape: (1097, 8)


## Tokenize the data

In [8]:
print('Vectorizing sequence data...')
tok = Tokenizer(num_words=100000)
tok.fit_on_texts(x_train80_txt)
# summarize what was learned
print(tok.document_count)

Vectorizing sequence data...
4388


In [9]:
len(tok.word_counts)

18030

In [19]:
x_train80 = tok.texts_to_sequences(x_train80_txt)
x_val = tok.texts_to_sequences(x_val_txt)

In [11]:
'''Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
'''
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

max_words = 1000
batch_size = 32
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Loading data...
8982 train sequences
2246 test sequences
46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)
Building model...
Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 5s - loss: 1.4289 - acc: 0.6792 - val_loss: 1.0905 - val_acc: 0.7631
Epoch 2/5
8083/8083 [==============================] - 4s - loss: 0.7893 - acc: 0.8183 - val_loss: 0.9371 - val_acc: 0.7898
Epoch 3/5
8083/8083 [==============================] - 4s - loss: 0.5505 - acc: 0.8649 - val_loss: 0.8926 - val_acc: 0.8009
Epoch 4/5
8083/8083 [==============================] - 4s - loss: 0.4176 - acc: 0.8993 - val_loss: 0.8807 - val_acc: 0.8098
Epoch 5/5
1824/2246 [=======================>......] - ETA: 0sTest score: 0.887346528199
Test accuracy: 0.793855743544
